# Tutorial 04: Function Calling

**Objetivos de Aprendizagem:**
- Entender o conceito de Function Calling na API OpenAI
- Aprender a definir funções para o modelo chamar
- Processar respostas com tool calls
- Implementar agentes que executam ações externas
- Criar sistemas que integram GPT com funções Python

**Pré-requisitos:**
- Tutorial 01: Conversa Básica concluído
- Tutorial 02: Agentes Especializados (recomendado)
- Conhecimento básico de Python e JSON

---


## Introdução

Function Calling permite que o GPT chame funções Python que você define, criando agentes capazes de executar ações no mundo real. O modelo decide quando e quais funções chamar baseado na conversa.

### Por que usar Function Calling?

1. **Ações Externas**: Permite ao GPT executar ações (buscar dados, calcular, enviar e-mails)
2. **Dados em Tempo Real**: Acessa informações atualizadas (preços, clima, notícias)
3. **Agentes Inteligentes**: Cria assistentes que fazem mais que apenas conversar
4. **Integração com Sistemas**: Conecta GPT com APIs, bancos de dados, serviços externos

### Conceitos-Chave

**Tools (Ferramentas):**
- Funções que o modelo pode chamar
- Definidas em formato JSON Schema
- Incluem nome, descrição e parâmetros

**Tool Calls:**
- Quando o modelo decide chamar uma função
- Retorna nome da função e argumentos
- Você executa a função e retorna o resultado

**Tool Choice:**
- `auto`: Modelo decide quando chamar (padrão)
- `none`: Nunca chama funções
- `required`: Sempre chama uma função
- `{"type": "function", "function": {"name": "nome"}}`: Força função específica

**Fluxo de Function Calling:**
1. Definir funções disponíveis
2. Enviar mensagem com `tools` definido
3. Verificar se há `tool_calls` na resposta
4. Executar função(s) chamada(s)
5. Enviar resultado de volta ao modelo
6. Receber resposta final do modelo

**Modelos Recomendados:**
- `gpt-3.5-turbo`: Suporta function calling
- `gpt-4`: Melhor para funções complexas
- `gpt-4o-mini`: Econômico e eficiente

---


## Configuração Inicial


In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

# Carregar variáveis de ambiente
load_dotenv()

# Inicializar cliente OpenAI
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

print("Cliente OpenAI configurado com sucesso!")


## Exemplo 1: Function Calling Básico

Criando uma função simples que o modelo pode chamar para obter informações.

**Quando usar:**
- Buscar dados externos
- Realizar cálculos
- Acessar APIs
- Consultar bancos de dados

**Vantagens:**
- Modelo decide quando usar a função
- Integração natural com código Python
- Respostas baseadas em dados reais


In [ ]:
# Definir função Python que será chamada pelo modelo
def obter_clima(cidade: str) -> str:
    """
    Obtém informações do clima de uma cidade.
    
    Args:
        cidade: Nome da cidade
    
    Returns:
        str: Informações do clima
    """
    # Simulação de dados do clima (em produção, chamaria uma API real)
    dados_clima = {
        "São Paulo": "25°C, ensolarado, umidade 60%",
        "Rio de Janeiro": "28°C, parcialmente nublado, umidade 70%",
        "Brasília": "22°C, chuvoso, umidade 80%",
    }
    return dados_clima.get(cidade, f"Clima de {cidade}: dados não disponíveis")

# Definir a função no formato que o modelo entende (JSON Schema)
funcoes_disponiveis = [
    {
        "type": "function",
        "function": {
            "name": "obter_clima",
            "description": "Obtém informações do clima de uma cidade específica",
            "parameters": {
                "type": "object",
                "properties": {
                    "cidade": {
                        "type": "string",
                        "description": "Nome da cidade para consultar o clima"
                    }
                },
                "required": ["cidade"]
            }
        }
    }
]

# Mapeamento de nomes de funções para funções Python
mapeamento_funcoes = {
    "obter_clima": obter_clima
}

print("Funções configuradas:")
print("- obter_clima(cidade)")


### Processar Function Call

Agora vamos fazer uma requisição e processar a resposta, executando a função se necessário.


In [ ]:
# Mensagens da conversa
mensagens = [
    {
        "role": "system",
        "content": "Você é um assistente útil que pode consultar informações do clima."
    },
    {
        "role": "user",
        "content": "Qual é o clima em São Paulo hoje?"
    }
]

# Fazer requisição com function calling
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=mensagens,
    tools=funcoes_disponiveis,
    tool_choice="auto"  # Modelo decide se chama a função
)

mensagem_resposta = response.choices[0].message

# Verificar se o modelo quer chamar alguma função
if mensagem_resposta.tool_calls:
    print("🔧 Modelo quer chamar função(s):")
    print()
    
    # Adicionar mensagem do assistente ao histórico
    mensagens.append(mensagem_resposta)
    
    # Processar cada tool call
    for tool_call in mensagem_resposta.tool_calls:
        nome_funcao = tool_call.function.name
        argumentos = json.loads(tool_call.function.arguments)
        tool_call_id = tool_call.id
        
        print(f"  Função: {nome_funcao}")
        print(f"  Argumentos: {argumentos}")
        print()
        
        # Executar função
        funcao = mapeamento_funcoes[nome_funcao]
        resultado = funcao(**argumentos)
        
        print(f"  Resultado: {resultado}")
        print()
        
        # Adicionar resultado ao histórico
        mensagens.append({
            "role": "tool",
            "tool_call_id": tool_call_id,
            "name": nome_funcao,
            "content": resultado
        })
    
    # Obter resposta final do modelo com o resultado da função
    resposta_final = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=mensagens
    )
    
    print("💬 Resposta final do assistente:")
    print("-" * 60)
    print(resposta_final.choices[0].message.content)
else:
    # Resposta direta sem chamar função
    print("💬 Resposta do assistente:")
    print("-" * 60)
    print(mensagem_resposta.content)


## Exemplo 2: Múltiplas Funções

Criando um assistente com várias funções disponíveis.

**Quando usar:**
- Agentes com múltiplas capacidades
- Sistemas que precisam de diferentes ações
- Integração com vários serviços

**Vantagens:**
- Modelo escolhe a função mais adequada
- Pode chamar múltiplas funções em sequência
- Flexibilidade para diferentes tarefas


In [ ]:
# Definir múltiplas funções
def calcular(expressao: str) -> str:
    """
    Calcula o resultado de uma expressão matemática.
    
    Args:
        expressao: Expressão matemática (ex: "2 + 2", "10 * 5")
    
    Returns:
        str: Resultado do cálculo
    """
    try:
        resultado = eval(expressao)
        return f"Resultado: {resultado}"
    except Exception as e:
        return f"Erro no cálculo: {e}"

def converter_moeda(valor: float, de: str, para: str) -> str:
    """
    Converte valor entre moedas.
    
    Args:
        valor: Valor a converter
        de: Moeda de origem (BRL, USD, EUR)
        para: Moeda de destino (BRL, USD, EUR)
    
    Returns:
        str: Valor convertido
    """
    # Taxas de câmbio simuladas (em produção, usar API real)
    taxas = {
        "BRL": {"USD": 0.20, "EUR": 0.18},
        "USD": {"BRL": 5.00, "EUR": 0.90},
        "EUR": {"BRL": 5.55, "USD": 1.11}
    }
    
    if de == para:
        return f"{valor} {de}"
    
    taxa = taxas.get(de, {}).get(para)
    if not taxa:
        return f"Conversão de {de} para {para} não disponível"
    
    resultado = valor * taxa
    return f"{valor} {de} = {resultado:.2f} {para}"

# Definir todas as funções
funcoes_multiplas = [
    {
        "type": "function",
        "function": {
            "name": "calcular",
            "description": "Calcula o resultado de uma expressão matemática",
            "parameters": {
                "type": "object",
                "properties": {
                    "expressao": {
                        "type": "string",
                        "description": "Expressão matemática a calcular (ex: '2 + 2', '10 * 5')"
                    }
                },
                "required": ["expressao"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "converter_moeda",
            "description": "Converte valor entre diferentes moedas (BRL, USD, EUR)",
            "parameters": {
                "type": "object",
                "properties": {
                    "valor": {
                        "type": "number",
                        "description": "Valor a ser convertido"
                    },
                    "de": {
                        "type": "string",
                        "description": "Moeda de origem (BRL, USD, EUR)",
                        "enum": ["BRL", "USD", "EUR"]
                    },
                    "para": {
                        "type": "string",
                        "description": "Moeda de destino (BRL, USD, EUR)",
                        "enum": ["BRL", "USD", "EUR"]
                    }
                },
                "required": ["valor", "de", "para"]
            }
        }
    }
]

mapeamento_multiplas = {
    "calcular": calcular,
    "converter_moeda": converter_moeda
}

print("Funções disponíveis:")
print("- calcular(expressao)")
print("- converter_moeda(valor, de, para)")


In [ ]:
# Exemplo de uso com múltiplas funções
mensagens = [
    {
        "role": "system",
        "content": "Você é um assistente que pode fazer cálculos e conversões de moeda."
    },
    {
        "role": "user",
        "content": "Quanto é 15 * 8? E depois converta 100 dólares para reais."
    }
]

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=mensagens,
    tools=funcoes_multiplas,
    tool_choice="auto"
)

mensagem_resposta = response.choices[0].message

if mensagem_resposta.tool_calls:
    mensagens.append(mensagem_resposta)
    
    for tool_call in mensagem_resposta.tool_calls:
        nome_funcao = tool_call.function.name
        argumentos = json.loads(tool_call.function.arguments)
        tool_call_id = tool_call.id
        
        print(f"🔧 Chamando: {nome_funcao}({argumentos})")
        
        funcao = mapeamento_multiplas[nome_funcao]
        resultado = funcao(**argumentos)
        
        print(f"   Resultado: {resultado}")
        print()
        
        mensagens.append({
            "role": "tool",
            "tool_call_id": tool_call_id,
            "name": nome_funcao,
            "content": resultado
        })
    
    # Se houver múltiplas tool calls, o modelo pode querer chamar mais funções
    # Continuar até não haver mais tool calls
    while True:
        resposta = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=mensagens
        )
        
        mensagem = resposta.choices[0].message
        mensagens.append(mensagem)
        
        if not mensagem.tool_calls:
            break
        
        for tool_call in mensagem.tool_calls:
            nome_funcao = tool_call.function.name
            argumentos = json.loads(tool_call.function.arguments)
            tool_call_id = tool_call.id
            
            print(f"🔧 Chamando: {nome_funcao}({argumentos})")
            
            funcao = mapeamento_multiplas[nome_funcao]
            resultado = funcao(**argumentos)
            
            print(f"   Resultado: {resultado}")
            print()
            
            mensagens.append({
                "role": "tool",
                "tool_call_id": tool_call_id,
                "name": nome_funcao,
                "content": resultado
            })
    
    print("💬 Resposta final:")
    print("-" * 60)
    print(mensagens[-1].content)
else:
    print("💬 Resposta:")
    print("-" * 60)
    print(mensagem_resposta.content)


## Exemplo 3: Função Auxiliar para Function Calling

Criando uma função auxiliar que simplifica o processamento de function calls.

**Vantagens:**
- Código mais limpo e reutilizável
- Tratamento de erros centralizado
- Facilita manutenção


In [ ]:
def processar_function_call(mensagens, funcoes, mapeamento_funcoes, max_iteracoes=5):
    """
    Processa function calls automaticamente até obter resposta final.
    
    Args:
        mensagens: Lista de mensagens da conversa
        funcoes: Lista de funções disponíveis (formato OpenAI)
        mapeamento_funcoes: Dict mapeando nome da função para função Python
        max_iteracoes: Número máximo de iterações (evita loops infinitos)
    
    Returns:
        str: Resposta final do modelo
    """
    iteracoes = 0
    
    while iteracoes < max_iteracoes:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=mensagens,
            tools=funcoes,
            tool_choice="auto"
        )
        
        mensagem = response.choices[0].message
        mensagens.append(mensagem)
        
        # Se não há tool calls, retornar resposta final
        if not mensagem.tool_calls:
            return mensagem.content
        
        # Processar cada tool call
        for tool_call in mensagem.tool_calls:
            nome_funcao = tool_call.function.name
            argumentos = json.loads(tool_call.function.arguments)
            tool_call_id = tool_call.id
            
            # Executar função
            if nome_funcao in mapeamento_funcoes:
                funcao = mapeamento_funcoes[nome_funcao]
                try:
                    resultado = funcao(**argumentos)
                except Exception as e:
                    resultado = f"Erro ao executar função: {e}"
            else:
                resultado = f"Função '{nome_funcao}' não encontrada"
            
            # Adicionar resultado ao histórico
            mensagens.append({
                "role": "tool",
                "tool_call_id": tool_call_id,
                "name": nome_funcao,
                "content": str(resultado)
            })
        
        iteracoes += 1
    
    return "Número máximo de iterações atingido"

print("✅ Função auxiliar criada!")


In [ ]:
# Exemplo usando a função auxiliar
mensagens = [
    {
        "role": "system",
        "content": "Você é um assistente que pode fazer cálculos."
    },
    {
        "role": "user",
        "content": "Calcule (25 + 15) * 3 e depois divida o resultado por 4"
    }
]

resposta = processar_function_call(
    mensagens=mensagens,
    funcoes=funcoes_multiplas,
    mapeamento_funcoes=mapeamento_multiplas
)

print("💬 Resposta:")
print("-" * 60)
print(resposta)


## Boas Práticas

### 1. Definição de Funções
- **Descrições claras**: A descrição da função ajuda o modelo a decidir quando usá-la
- **Parâmetros específicos**: Use `enum` para valores limitados, `type` correto para cada parâmetro
- **Campos obrigatórios**: Defina `required` apenas para parâmetros essenciais
- **Nomes descritivos**: Nomes de funções devem ser claros e autoexplicativos

### 2. Tratamento de Erros
- Sempre valide argumentos antes de executar
- Trate exceções e retorne mensagens de erro claras
- Implemente fallbacks para funções que podem falhar
- Log de erros para debugging

### 3. Segurança
- **Nunca use `eval()` em produção** sem validação rigorosa
- Valide todos os inputs antes de processar
- Limite o que as funções podem fazer
- Implemente rate limiting se necessário

### 4. Performance
- Funções devem ser rápidas (evite operações lentas)
- Use cache quando apropriado
- Limite número de iterações para evitar loops infinitos
- Monitore uso de tokens

### 5. Tool Choice
- `auto`: Use na maioria dos casos (modelo decide)
- `none`: Quando não quer que o modelo chame funções
- `required`: Quando sempre precisa de uma função
- Função específica: Quando quer forçar uma função

---


## Exercícios Práticos

### Exercício 1: Criar Função de Busca
Crie uma função que busca informações sobre produtos em um catálogo e integre com Function Calling.

### Exercício 2: Agente com Múltiplas Funções
Crie um agente que pode fazer cálculos, converter moedas e buscar informações, escolhendo a função adequada automaticamente.

### Exercício 3: Sistema de Agendamento
Implemente um sistema que permite ao modelo agendar eventos, verificando disponibilidade e criando compromissos.

---


## Referências

- [Documentação - Function Calling](https://platform.openai.com/docs/guides/function-calling)
- [API Reference - Chat Completions](https://platform.openai.com/docs/api-reference/chat/create)
- [JSON Schema](https://json-schema.org/)
- [Best Practices](https://platform.openai.com/docs/guides/function-calling)
